# Validation script for Imagenet models
Use this notebook to verify accuracy of a trained model on validation set. Works for all models trained on Imagenet dataset ([ILSVRC2012](http://www.image-net.org/challenges/LSVRC/2012/) and onwards)

## Guidelines for usage
* The MXNet symbol (.json) and params (.params) file for the model must be in the root folder
* Dependencies are same as that for training notebook. Verify by running the cell below
* Imagenet dataset must be downloaded and extracted in the required directory structure. Can use [imagenet.py]() for this

In [8]:
import matplotlib
matplotlib.use('Agg')

import mxnet as mx
import numpy as np
from mxnet import gluon, nd
from mxnet.gluon.data.vision import transforms

from gluoncv.data import imagenet
from collections import namedtuple

In [2]:
# choose context as cpu or gpu(recommended)
# ctx = [mx.cpu()]
ctx = [mx.gpu(0)]

# path to imagenet dataset folder
data_dir = '../imagenet/img_dataset'

# batch size (set to 1 for cpu)
batch_size = 128

# number of preprocessing workers
num_workers = 32

# model name
model_name = 'vgg16'

In [3]:
# Load symbols and params file
sym, arg_params, aux_params = mx.model.load_checkpoint(model_name, 0)

In [4]:
# Define model
mod = mx.mod.Module(symbol=sym, context=ctx, label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [5]:
# Define evaluation metrics
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)

# Define image transforms
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Prepare input
val_data = gluon.data.DataLoader(
    imagenet.classification.ImageNet(data_dir, train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [6]:
# Compute evaluations
Batch = namedtuple('Batch', ['data'])
acc_top1.reset()
acc_top5.reset()
num_batches = int(50000/batch_size)
print('[0 / %d] batches done'%(num_batches))
for i, batch in enumerate(val_data):
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
    mod.forward(Batch([data[0]]))
    outputs=mod.get_outputs()
    acc_top1.update(label, outputs)
    acc_top5.update(label, outputs)
    if (i+1)%50==0:
        print('[%d / %d] batches done'%(i+1,num_batches))

[0 / 390] batches done
[50 / 390] batches done
[100 / 390] batches done
[150 / 390] batches done
[200 / 390] batches done
[250 / 390] batches done
[300 / 390] batches done
[350 / 390] batches done


In [7]:
# Print results
_, top1 = acc_top1.get()
_, top5 = acc_top5.get()
print('top1 error:',1-top1,'; top-5 error:',1-top5)

top1 error: 0.28781999999999996 ; top-5 error: 0.09670000000000001
